In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options

options = Options()
# options.add_argument('--headless')
driver = webdriver.Edge(options=options)

driver.get("https://skycoach.gg/")

# Click the "Choose a game" button to open the menu
# choose_game_btn = driver.find_element(By.CSS_SELECTOR, ".header__menu-button button button--action button--gradient-violet")
choose_game_btn = driver.find_element(By.CSS_SELECTOR, ".header__menu-button")
choose_game_btn.click()

import time
time.sleep(2)  # Adjust if needed

menu_content = driver.find_element(By.CLASS_NAME, "menu__wrapper").get_attribute('innerHTML')
print(menu_content)

driver.quit()


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

soup = BeautifulSoup(menu_content, "html.parser")
games = []

for li in soup.find_all("li", class_="menu__game-section-item"):
    a_tag = li.find("a", class_="menu__game-section-item-link")
    if a_tag:
        name = a_tag.find("span").get_text(strip=True)
        href = a_tag.get("href")
        img_tag = a_tag.find("img", class_="menu__game-section-item-image")
        img_src = img_tag.get("src") if img_tag else None
        games.append({"Game Name": name, "Link": href, "Image": img_src})

df_games = pd.DataFrame(games)
print(df_games)


In [ ]:
df_games.to_csv("games.csv", index=False)


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
import time
import os
import requests
from bs4 import BeautifulSoup
options = Options()
options.add_argument('--headless')
driver = webdriver.Edge(options=options)

# Load the CSV file and display its contents
df = pd.read_csv("games.csv")

all_services = []

for index, row in df.iterrows():
    game_name = row['Game Name']
    game_link = row['Link']
    print(f"Fetching data for {game_name} from {game_link}")
    if ":" in game_name:
        game_name = game_name.replace(":", "-")
    complete_game_link = "https://skycoach.gg" + game_link
    print(f"Complete game link: {complete_game_link}")
    # Fetch the HTML content of the game page
    page_source = requests.get(complete_game_link).text
    time.sleep(2)  # Wait for the page to load

    game_page_content = page_source
    soup = BeautifulSoup(game_page_content, "html.parser")
    sidebar = soup.find("div", class_="game-tags-sidebar__categories")
    if sidebar:
        # Find all main categories
        for cat in sidebar.find_all("div", class_="game-tags-sidebar__category", recursive=True):
            head = cat.find("div", class_="game-tags-sidebar__category-head")
            print(head)
            if head:
                a_tag = head.find("a", class_="game-tags-sidebar__category-title")
                if a_tag:
                    print(a_tag)
                    service_name = a_tag.get_text(strip=True)
                    service_link = a_tag.get("href")
                    all_services.append({
                        "Game Name": game_name,
                        "Category": service_name,
                        "Link": "https://skycoach.gg" + service_link if service_link else ""
                    })
            # Check for subcategories
            subcats = cat.find("div", class_="game-tags-sidebar__subcategories")
            if subcats:
                print(subcats)
                for sub_a in subcats.find_all("a", class_="game-tags-sidebar__subcategory"):
                    sub_name = sub_a.get_text(strip=True)
                    sub_link = sub_a.get("href")
                    all_services.append({
                        "Game Name": game_name,
                        "Category": service_name,
                        "Subcategory": sub_name,
                        "Link": "https://skycoach.gg" + sub_link if sub_link else ""
                    })
    print(f"Processed sidebar for {game_name}")
    df_services = pd.DataFrame(all_services)
    # check if folder exists
    if not os.path.exists("gamesTwo"):
        os.makedirs("gamesTwo")
    df_services.to_csv(f"gamesTwo/{game_name.replace(' ', '_')}_services.csv", index=False)
    all_services = []  # Reset for the next game
    time.sleep(2)  # Adjust sleep time as needed
    
    # Remove break if you want to process all games


# Convert to DataFrame and save to CSV
    

In [ ]:
import json

api_url = "https://skycoach.gg/api/shop/product/list/badge-products?badge_id=3&fields_data=compact"
response = requests.get(api_url)
data = response.json()
data_keys = data.keys()
print(data)
# Create a sample payload structure with "" values for each field
# sample_payload = {}
# for key in data_keys:
#     if isinstance(data[key], list) and data[key]:
#         sample_payload[key] = {field: "" for field in data[key][0].keys()}
#     else:
#         sample_payload[key] = None
# print(data)

# Save the sample payload to a JSON file
# with open("sample_payload.json", "w") as f:
#     json.dump(sample_payload, f, indent=4)

# save data in text file.txt
with open("data.txt", "w") as f:
    json.dump(data, f, indent=4)

# print(sample_payload)

for key in data_keys:
    print(f"Key: {key}")
    for item in data[key]:
        title = item.get("title", "No title")
        price = item.get("price", "No price")
        print(f"Title: {title}, Price: {price}")
# for item in data["2"]:
#     title = item.get("title", "No title")
#     price = item.get("price", "No price")
#     print(f"Title: {title}, Price: {price}")

In [ ]:
import os

games_files = set(os.listdir("games"))
gamesTwo_files = set(os.listdir("gamesTwo"))

# Find files present in gamesTwo but not in games
extra_in_gamesTwo = gamesTwo_files - games_files
# Find files present in games but not in gamesTwo
extra_in_games = games_files - gamesTwo_files

print("Extra files in gamesTwo:")
for fname in extra_in_gamesTwo:
    print(fname)

print("\nExtra files in games:")
for fname in extra_in_games:
    print(fname)

In [ ]:
# https://www.skycoach.gg/storage/uploads/game-assets/icons/8a9fa179ac.svg

# save this url svg in local directory /uploads
def save_svg_image(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
# Example usage
save_svg_image("https://www.skycoach.gg/storage/uploads/game-assets/icons/254932daa2.svg", "game_icon.svg")

In [ ]:
import pandas as pd
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.edge.options import Options
# import time
import os
# import requests
# from bs4 import BeautifulSoup
# options = Options()
# options.add_argument('--headless')
# driver = webdriver.Edge(options=options)
# /get all file names from gamesTwo directory
gamesTwo_files = os.listdir("gamesTwo")


# Loop through each file in the directory
for file_name in gamesTwo_files:
    file_name = "RBLX_services.csv"
    if file_name.endswith(".csv"):
        # Read the CSV file
        df = pd.read_csv(f"./gamesTwo/{file_name}")
        
        # Check if the 'Link' column exists
        if 'Link' in df.columns:
            # Loop through each row in the DataFrame
            for index, row in df.iterrows():
                link = row['Link']
                if link and link.startswith("https://skycoach.gg/"):
                    # extract data from link
                    print(f"Processing link: {link}")
                    # Here you can add your logic to process the link
                    page_source = requests.get(link).text
                    print(page_source)
                    """<div class="game-header" data-v-265d981e data-v-059b5164><div class="game-header__top-container" data-v-265d981e><h1 class="h1 sm:h1-mobile" data-v-265d981e>
      8 Ball Pool Coins
    </h1> <!----></div> <!----></div>
    
    <picture class="responsive-image offer-card__image" data-v-26f85f9a data-v-0d6f92b4><source media="(-webkit-min-device-pixel-ratio: 2)" type="" srcset="/storage/uploads/products/8-ball-pool-coins1717452870_picture_item.png" data-v-26f85f9a> <img src="/storage/uploads/products/8-ball-pool-coins1717452871_picture_item_small.png" data-fallback-img="/images/games/picture_item.png" class="responsive-image__img" data-v-26f85f9a></picture> 
 
       <div class="product-info-sections product-offer__description" data-v-3d474cdb data-v-059b5164><div class="product-info-section" data-v-3d474cdb><h2 class="product-info-section__title h2" data-v-3d474cdb>
      Description
    </h2> <div class="product-info-section__content" data-v-3d474cdb><div class="product-info-section__wrapper" data-v-3d474cdb><div class="product-info-section__html text-medium wysiwyg-content" data-v-3d474cdb><p><span>Buy cheap 8 Ball Pool Coins&nbsp;on our platform and be ahead of everyone else.</span></p>
<p><span>In 8 Ball Pool, players can use special props like more accurate aiming lines and power-enhancing items. By acquiring enough 8 Ball Pool coins, players can purchase these props to gain additional advantages and improve their gameplay. So, there is never enough of this valuable currency, and getting it is a very tiring and time-consuming process. So, why waste your precious time when you can quickly get our 8 Ball Pool Coins for sale?</span></p>
<p><span>Contact us now and get as many 8 Ball Pool Coins as you need with maximum safety and quick delivery!</span></p>
<ol>
<li><span>Selected amount of 8 Ball Pool Coins;</span></li>
<li><span>100% safety;</span></li>
<li><span>Complete safety and 24/7 support.</span></li>
</ol></div></div></div></div> <div class="product-info-section" data-v-3d474cdb><h2 class="product-info-section__title h2" data-v-3d474cdb>
      Requirements
    </h2> <div class="product-info-section__content" data-v-3d474cdb><div class="product-info-section__wrapper" data-v-3d474cdb><div class="product-info-section__html text-medium wysiwyg-content" data-v-3d474cdb><ul>
<li>Active account;</li>
<li>20+ level (Semi-pro Rank).</li>
</ul></div></div></div></div> <div class="product-info-section" data-v-3d474cdb><h2 class="product-info-section__title h2" data-v-3d474cdb>
      More info
    </h2> <div class="product-info-section__content" data-v-3d474cdb><div class="product-info-section__wrapper" data-v-3d474cdb><div class="product-info-section__html text-medium wysiwyg-content" data-v-3d474cdb><p><span>HOW DO WE DELIVER 8 BALL POOL COINS?</span></p>
<p>You can get cheap 8 Ball Pool Coins via the following completion method:</p>
<p><span>Comfort Trade:</span></p>
<ul>
<li><span>This is the quickest and the most reliable 8 Ball Pool Coins delivery method. Our pro-player will safely access your account and carry out your order. All security measures will be in constant use so that you can rest assured - your account is in good hands.&nbsp;</span></li>
</ul></div></div></div></div></div>



<a href="/8-ball-pool-boost" class="breadcrumbs__back-button router-link-active button button--secondary button--white-transparent-10" data-v-f2586174 data-v-a5397112><svg aria-hidden="true" class="ui-icon breadcrumbs__back-button-icon" data-v-039e196d data-v-a5397112><use href="/images/icons/sprite.svg?v=14#chevron-left" data-v-039e196d></use></svg> <span class="breadcrumbs__back-button-text" data-v-a5397112>Coins</span></a>
 
<div class="payment-summary__price" data-v-0ce13794><!----> <div class="payment-summary__price-row payment-summary__price-row-total" data-v-0ce13794><div class="payment-summary__label-column" data-v-0ce13794><span class="payment-summary__label-total text-medium" data-v-0ce13794>
          Total
        </span> <!----></div> <div class="payment-summary__price-column" data-v-0ce13794><!----> <span class="payment-summary__price-column-total h2" data-v-0ce13794>
          0,99 €
        </span></div></div></div>     
       
             """

    break

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
results = []
gamesTwo_files = list(os.listdir("gamesTwo"))
for file_name in gamesTwo_files:
  if file_name.endswith(".csv"):
    print(file_name)
    df = pd.read_csv(f"gamesTwo/{file_name}")
    category = file_name.replace("_services.csv", "").replace("_", " ")
    if 'Link' in df.columns:
      for index, row in df.iterrows():
        link = row['Link']
        if link and link.startswith("https://skycoach.gg/"):
          print(f"Processing link: {link}")
          page_source = requests.get(link).text
          soup = BeautifulSoup(page_source, "html.parser")
          
          # Name
          name_tag = soup.find("div", class_="game-header")
          name = None
          if name_tag:
            h1 = name_tag.find("h1")
            if h1:
              name = h1.get_text(strip=True)
          
          # Description
          desc = None
          desc_section = soup.find("div", class_="product-info-section__html")
          if desc_section:
            desc = desc_section.get_text(separator="\n", strip=True)
          
          # Icon
          icon = None
          picture = soup.find("picture", class_="responsive-image")
          if picture:
            img = picture.find("img")
            if img and img.get("src"):
              icon = img["src"]
              if icon.startswith("/"):
                icon = "https://skycoach.gg" + icon
          
          # Price
          price = None
          price_span = soup.find("span", class_="payment-summary__price-column-total")
          if price_span:
            price = price_span.get_text(strip=True)
          
          # Category
          cat = row.get("Category", category)
          
          results.append({
            "Name": name,
            "Description": desc,
            "Icon": icon,
            "Price": price,
            "Category": cat,
            "Link": link
          })
    break

df_result = pd.DataFrame(results)
print(df_result)


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os
results = []
gamesTwo_files = set(os.listdir("gamesTwo"))
for file_name in gamesTwo_files:
  if file_name.endswith(".csv"):
    print(file_name)
    df = pd.read_csv(f"gamesTwo/{file_name}")
    category = file_name.replace("_services.csv", "").replace("_", " ")
    if 'Link' in df.columns:
      for index, row in df.iterrows():
        link = row['Link']
        if link and link.startswith("https://skycoach.gg/"):
          print(f"Processing link: https://skycoach.gg/roblox-boost{link}")
          page_source = requests.get("https://skycoach.gg/roblox-boost").text
          soup = BeautifulSoup(page_source, "html.parser")
          
        #   # Name
        #   name_tag = soup.find("div", class_="game-header")
        #   name = None
        #   if name_tag:
        #     h1 = name_tag.find("h1")
        #     if h1:
        #       name = h1.get_text(strip=True)
          
        #   # Description
        #   desc = None
        #   desc_section = soup.find("div", class_="product-info-section__html")
        #   if desc_section:
        #     desc = desc_section.get_text(separator="\n", strip=True)
          
        #   # Icon
        #   icon = None
        #   picture = soup.find("picture", class_="responsive-image")
        #   if picture:
        #     img = picture.find("img")
        #     if img and img.get("src"):
        #       icon = img["src"]
        #       if icon.startswith("/"):
        #         icon = "https://skycoach.gg" + icon
          
        #   # Price
        #   price = None
        #   price_span = soup.find("span", class_="payment-summary__price-column-total")
        #   if price_span:
        #     price = price_span.get_text(strip=True)
          
        #   # Category
        #   cat = row.get("Category", category)
          
        #   results.append({
        #     "Name": name,
        #     "Description": desc,
        #     "Icon": icon,
        #     "Price": price,
        #     "Category": cat,
        #     "Link": link
        #   })
    break

df_result = pd.DataFrame(results)
print(df_result)


MoCo_services.csv
Processing link: https://skycoach.gg/moco-boost/leveling
                  Name Description  \
0  MoCo Leveling Boost        None   

                                                Icon Price  Category  \
0  https://skycoach.gg/storage/uploads/game-asset...  None  Leveling   

  Subcategory                                     Link  
0        None  https://skycoach.gg/moco-boost/leveling  


In [9]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os
results = []
gamesTwo_files = set(os.listdir("gamesTwo"))
for file_name in gamesTwo_files:
  if file_name.endswith(".csv"):
    print(file_name)
    df = pd.read_csv(f"gamesTwo/{file_name}")
    category = file_name.replace("_services.csv", "").replace("_", " ")
    if 'Link' in df.columns:
      for index, row in df.iterrows():
        print(row)
        subcat = row.get("Subcategory", "")
        if subcat:
          print(f"Processing subcategory: {subcat}")
        else:
          print(f"Processing category: {category}")
        link = row['Link']
        if link and link.startswith("https://skycoach.gg/"):
          print(f"Processing link: {link}")
          page_source = requests.get("https://skycoach.gg/path-of-exile-2-boost/products/exalted-orbs-6477").text
          soup = BeautifulSoup(page_source, "html.parser")
          asides = soup.find_all("aside", class_="game-tags-sidebar game-main-page-sidebar")
          soup2 = BeautifulSoup(str(asides), "html.parser")
          hrefs = [a.get("href") for a in soup2.find_all("a") if a.get("href")]

          # Name
          name_tag = soup.find("div", class_="game-header")
          name = None
          if name_tag:
            h1 = name_tag.find("h1")
            if h1:
              name = h1.get_text(strip=True)
          
          # Description
          desc = None
          desc_section = soup.find("div", class_="product-info-section__html")
          if desc_section:
            desc = desc_section.get_text(separator="\n", strip=True)
          
          # Icon
          icon = None
          picture = soup.find("picture", class_="responsive-image")
          if picture:
            img = picture.find("img")
            if img and img.get("src"):
              icon = img["src"]
              if icon.startswith("/"):
                icon = "https://skycoach.gg" + icon
          
          # Price
          price = None
          price_span = soup.find("span", class_="payment-summary__price-column-total")
          if price_span:
            price = price_span.get_text(strip=True)
          
          # Category
          cat = row.get("Category", category)
          
          results.append({
            "Name": name,
            "Description": desc,
            "Icon": icon,
            "Price": price,
            "Category": cat,
            "Link": link
          })
    break


print(hrefs)


Path_of_Exile_2_services.csv
Game Name                                      Path of Exile 2
Category                                      Dawn of the Hunt
Link         https://skycoach.gg/path-of-exile-2-boost/curr...
Name: 0, dtype: object
Processing category: Path of Exile 2
Processing link: https://skycoach.gg/path-of-exile-2-boost/current-league
Game Name                                      Path of Exile 2
Category                                         Powerleveling
Link         https://skycoach.gg/path-of-exile-2-boost/powe...
Name: 1, dtype: object
Processing category: Path of Exile 2
Processing link: https://skycoach.gg/path-of-exile-2-boost/powerleveling
Game Name                                      Path of Exile 2
Category                                              Currency
Link         https://skycoach.gg/path-of-exile-2-boost/curr...
Name: 2, dtype: object
Processing category: Path of Exile 2
Processing link: https://skycoach.gg/path-of-exile-2-boost/currency
Game Name

In [10]:
results

[{'Name': 'PoE 2 Exalted Orbs',
  'Description': 'Buy PoE 2 Exalted Orbs to receive any amount of this currency in no time! If you want to succeed in any ARPG, you will always need top-tier gear and lots of essential currencies to get this gear. Exalted Orbs are used to enhance a Rare item with a random modifier. Primarily, this is one of the main trading currencies that will help you buy anything you want, especially at the start of the League.\nHowever, due to low drop rates, there are not enough of them every time. To avoid the long-lasting and exhausting grinding, you can access PoE 2 Exalted Orbs for sale quickly and safely at Skycoach! You will get:\nThe chosen amount of PoE 2 Exalted Orbs for your character;\nQuick and easy trading for any purpose;\nAccess to modifying your Rare gear;\nDozens of hours saved.',
  'Icon': 'https://skycoach.gg/storage/uploads/products/poe-2-exalted-orbs1732807855_picture_bg.png',
  'Price': '0,99\xa0€',
  'Category': 'Dawn of the Hunt',
  'Link': '